<h1>Hosting GPT-2</h2>

In [4]:
import syft as sy
import torch as th
import grid as gr
from pytorch_transformers import GPT2Tokenizer, GPT2Model

In [3]:
hook = sy.TorchHook(th)

<h2>Set up Configs</h2>

In [6]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Load pre-trained model (weights)
model = GPT2Model.from_pretrained('gpt2')

<h2>Setting Input</h2>

In [8]:
# Encode a text inputs
text = "Who was Jim Henson ? Jim Henson was a"
indexed_tokens = tokenizer.encode(text)

# Convert indexed tokens in a PyTorch tensor
tokens_tensor = th.tensor([indexed_tokens])

<h2>Hosting GPT-2 Model</h2>

In [9]:
traced_model = th.jit.trace(model, (tokens_tensor,))

# Grid Node
bob = gr.WebsocketGridClient(hook, "http://localhost:3000/", id="Bob")
bob.connect()

# Host GPT-2 on Bob worker
bob.serve_model(traced_model, model_id="GPT-2")

/home/ionesio/workspace/venv/lib/python3.7/site-packages/syft-0.1.26a1-py3.7.egg/syft/frameworks/hook.py:342: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  response = method(*args, **kwargs)
/home/ionesio/workspace/venv/lib/python3.7/site-packages/syft-0.1.26a1-py3.7.egg/syft/frameworks/hook.py:342: TracerWarning: Converting a tensor to a Python index might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  response = method(*args, **kwargs)


{'success': True}

<h2>Runing Inference</h2>

In [12]:
%%time
response = bob.run_inference(model_id="GPT-2", data=tokens_tensor)

predictions = th.tensor(response['prediction'])
predicted_index = th.argmax(predictions[0, -1, :]).item()
predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])
print("Predicted text: ", predicted_text)

Predicted text:   Who was Jim Henson? Jim Henson was aage
CPU times: user 26.5 ms, sys: 0 ns, total: 26.5 ms
Wall time: 174 ms
